In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
from lightgbm import LGBMClassifier 

from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import TruncatedSVD

import plotly.express as px
import plotly.graph_objs as go
import statsmodels.formula.api as smf
from plotly.graph_objects import Layout

Import the dataset and do the required pre-processing

In [52]:
df = pd.read_csv("mergedData.csv")

In [53]:
df = df.drop("Unnamed: 0", axis = 1)

In [55]:
df = df.drop(["DriverNumber", "Driver", "Points"], axis = 1)
aux = df

In [56]:
df = df.drop(["AverageRPM"], axis = 1)
df = df.drop(["AvgLapTime"], axis = 1)
df = df.drop(["AvgSplitTime"], axis = 1)

In [57]:
# create new columns based on Status values
df['carIssue'] = (df['Status'] == 'carIssue').astype(int)
df['driverIssue'] = (df['Status'] == 'driverIssue').astype(int)
df = df.drop('Status', axis=1)

In [60]:
df["RaceCountry"] = df["RaceCountry"].str.replace(" ", "")
df["TeamName"] = df["TeamName"].str.replace(" ", "")
df["Engine"] = df["Engine"].str.replace(" ", "")

In [61]:
#df.to_csv(r'df_points.csv', index=True, header=True)
#df.to_csv(r'df_position.csv', index=True, header=True)
#df.to_csv(r'plotdata_points.csv', index=True, header=True)
#df.to_csv(r'plotdata_position.csv', index=True, header=True)

In [62]:
# get a list of all categorical variables except "Abbreviation" and "raceCountry"
cat_vars = df.select_dtypes(include=['object']).columns.tolist()
cat_vars.remove("Abbreviation")
cat_vars.remove("RaceCountry")

# create dummy variables
dummies = pd.get_dummies(df[cat_vars], drop_first=True)

# drop the original categorical variables from the DataFrame
df = df.drop(cat_vars, axis=1)

# concatenate the dummy variables with the remaining variables in the DataFrame
df = pd.concat([df, dummies], axis=1)


## MERF

In [70]:
from merf import MERF

In [75]:
# Split the data into two datasets based on the Year column
df_before = df[df['Year'].isin([2019, 2020])]
df_after = df[df['Year'].isin([2021, 2022])]

## Before model

## Train

In [77]:
# Split the data into train and test sets based on the raceID column
train_df = df_before[(df_before['raceID'] >= 1) & (df_before['raceID'] <= 29)]
test_df = df_before[(df_before['raceID'] >= 30) & (df_before['raceID'] <= 37)]

In [78]:
X_train = train_df.drop('Position', axis=1)
Y_train = train_df['Position']
X_test = test_df.drop('Position', axis=1)
Y_test = test_df['Position']

In [79]:
X_train = X_train.drop('RaceCountry', axis=1)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('RaceCountry', axis=1)
X_test = X_test.drop('Year', axis=1)

In [80]:
x_train = X_train.drop(['Abbreviation', 'raceID'], axis = 1)
Z_train = X_train[['raceID']]
clusters_train = X_train['raceID']

In [81]:
from sklearn.svm import SVR

svr_model = SVR(kernel='rbf', gamma='scale')

mrf = MERF(fixed_effects_model=svr_model, max_iterations=30)
mrf.fit(x_train, Z_train, clusters_train, Y_train)

INFO     [merf.py:307] Training GLL is 2435.012355347997 at iteration 1.
INFO     [merf.py:307] Training GLL is 2395.2879496758464 at iteration 2.
INFO     [merf.py:307] Training GLL is 2382.0263857090836 at iteration 3.
INFO     [merf.py:307] Training GLL is 2372.2077972567336 at iteration 4.
INFO     [merf.py:307] Training GLL is 2364.401558292814 at iteration 5.
INFO     [merf.py:307] Training GLL is 2358.022992796393 at iteration 6.
INFO     [merf.py:307] Training GLL is 2352.725299922297 at iteration 7.
INFO     [merf.py:307] Training GLL is 2348.1462431501573 at iteration 8.
INFO     [merf.py:307] Training GLL is 2344.130624386462 at iteration 9.
INFO     [merf.py:307] Training GLL is 2340.554890117171 at iteration 10.
INFO     [merf.py:307] Training GLL is 2337.360798404492 at iteration 11.
INFO     [merf.py:307] Training GLL is 2334.452009283764 at iteration 12.
INFO     [merf.py:307] Training GLL is 2331.790672118035 at iteration 13.
INFO     [merf.py:307] Training GLL is 2329

In [82]:
train_preds = mrf.predict(x_train, Z_train, clusters_train)

In [83]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_train, train_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, train_preds)
r2 = r2_score(Y_train, train_preds)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 32.5883534311462
Test RMSE: 5.708620974556482
Test MAE: 4.940584915454136
Test R-squared: -0.0006222312438370903


In [84]:
actual_values = Y_test
predictions = train_preds
# Calculate the mean of actual values
mean_actual = sum(actual_values) / len(actual_values)

# Calculate SSE and SST
sse = sum((actual - pred) ** 2 for actual, pred in zip(actual_values, predictions))
sst = sum((actual - mean_actual) ** 2 for actual in actual_values)

# Calculate R-squared
r2 = 1 - (sse / sst)
r2

0.00028128207952660134

## Test

In [85]:
x_test = X_test.drop(['Abbreviation', 'raceID'], axis = 1)
Z_test = X_test[['raceID']]
clusters_test = X_test['raceID']

In [87]:
y_pred = mrf.predict(x_test, Z_test, clusters_test)

In [88]:
mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 31.23552633458314
Test RMSE: 5.588875229827836
Test MAE: 4.825901981369629
Test R-squared: 0.0006842910781438283


## After model

## Train

In [91]:
# Split the data into train and test sets based on the raceID column
train_df = df_after[(df_after['raceID'] >= 38) & (df_after['raceID'] <= 72)]
test_df = df_after[(df_after['raceID'] >= 73) & (df_after['raceID'] <= 80)]

In [92]:
X_train = train_df.drop('Position', axis=1)
Y_train = train_df['Position']
X_test = test_df.drop('Position', axis=1)
Y_test = test_df['Position']

In [93]:
X_train = X_train.drop('RaceCountry', axis=1)
X_train = X_train.drop('Year', axis=1)
X_test = X_test.drop('RaceCountry', axis=1)
X_test = X_test.drop('Year', axis=1)

In [94]:
x_train = X_train.drop(['Abbreviation', 'raceID'], axis = 1)
Z_train = X_train[['raceID']]
clusters_train = X_train['raceID']

In [ ]:
mrf.fit(x_train, Z_train, clusters_train, Y_train)

INFO     [merf.py:307] Training GLL is 2698.3374904130565 at iteration 1.
INFO     [merf.py:307] Training GLL is 2666.032114070161 at iteration 2.
INFO     [merf.py:307] Training GLL is 2660.537947490707 at iteration 3.
INFO     [merf.py:307] Training GLL is 2655.6251547166144 at iteration 4.
INFO     [merf.py:307] Training GLL is 2651.2173845561233 at iteration 5.
INFO     [merf.py:307] Training GLL is 2647.2953006595058 at iteration 6.
INFO     [merf.py:307] Training GLL is 2643.741791912943 at iteration 7.
INFO     [merf.py:307] Training GLL is 2640.5278139988404 at iteration 8.
INFO     [merf.py:307] Training GLL is 2637.577168035475 at iteration 9.
INFO     [merf.py:307] Training GLL is 2634.8414779653212 at iteration 10.
INFO     [merf.py:307] Training GLL is 2632.303225669452 at iteration 11.
INFO     [merf.py:307] Training GLL is 2629.934668593295 at iteration 12.
INFO     [merf.py:307] Training GLL is 2627.719330425508 at iteration 13.
INFO     [merf.py:307] Training GLL is 26

In [96]:
train_preds = mrf.predict(x_train, Z_train, clusters_train)

In [97]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_train, train_preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, train_preds)
r2 = r2_score(Y_train, train_preds)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)

Test MSE: 31.564983546760896
Test RMSE: 5.618272291973832
Test MAE: 4.850067647381201
Test R-squared: 0.0012309089697312148


## Test

In [98]:
x_test = X_test.drop(['Abbreviation', 'raceID'], axis = 1)
Z_test = X_test[['raceID']]
clusters_test = X_test['raceID']

In [99]:
y_pred = mrf.predict(x_test, Z_test, clusters_test)

In [100]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print the performance metrics
print("Test MSE:", mse)
print("Test RMSE:", rmse)
print("Test MAE:", mae)
print("Test R-squared:", r2)


Test MSE: 31.555037790631737
Test RMSE: 5.617387096384914
Test MAE: 4.853123115063056
Test R-squared: 7.414877490208305e-05
